# Zadanie tokenizacja

In [ ]:
# --- IMPORTOWANIE NARZĘDZI ---
import tensorflow_datasets as tfds  # do ściągania gotowych zbiorów danych, w tym imdb
import numpy as np                 # do podstawowych operacji na danych, np. tablicach
import re                          # do wyrażeń regularnych, czyli czyszczenia tekstu
import nltk                        # Natural Language Toolkit (nasza baza NLP)
from nltk.tokenize import word_tokenize  # konkretna funkcja do dzielenia tekstu na słowa

try:
    # ściągamy 'punkt', to są zasoby potrzebne do poprawnej tokenizacji przez NLTK
    nltk.download('punkt')
except:
    pass

# --- ŁADOWANIE ZBIORU DANYCH IMDB ---
(ds_train, ds_test), ds_info = tfds.load(
    'imdb_reviews',                   # jaki zbiór danych chcemy
    split=['train', 'test'],          # od razu dzielimy na trening i test
    shuffle_files=True,
    as_supervised=True,               # dane w formacie (tekst, etykieta)
    with_info=True)

# przerabiamy dane z tfds na zwykłe listy stringów i listę etykiet (0/1)
# dekodujemy tekst z binarnego formatu na czytelny utf8
X_train_imdb = [doc.decode('utf8') for doc, label in tfds.as_numpy(ds_train)]
y_train_imdb = [label for doc, label in tfds.as_numpy(ds_train)]
X_test_imdb = [doc.decode('utf8') for doc, label in tfds.as_numpy(ds_test)]
y_test_imdb = [label for doc, label in tfds.as_numpy(ds_test)]

print(f"Dane treningowe: {len(X_train_imdb)} recenzji")
print(f"Dane testowe: {len(X_test_imdb)} recenzji")

# --- KLUCZOWA FUNKCJA: CZYSZCZENIE I TOKENIZACJA ---
def clean_and_tokenize(text):
    # 1. CZYSZCZENIE: usuwamy wszystko co nie jest literą (a-z, A-Z) ani spacją, zastępując spacją.
    # pozbywamy się interpunkcji i znaków specjalnych
    text = re.sub(r'[^a-zA-Z\s]', ' ', text)

    # 2. NORMALIZACJA: wszystko na małe litery, żeby 'Film' i 'film' były takie same
    text = text.lower()

    # 3. CZYSZCZENIE SPACJI: usuwamy wielokrotne spacje i spacje z początku/końca
    text = re.sub(r'\s+', ' ', text).strip()

    # 4. TOKENIZACJA: dzielimy tekst na listę pojedynczych słów (tokenów)
    tokens = word_tokenize(text)

    return tokens

# --- APLIKACJA FUNKCJI NA DANYCH ---
# puszczamy naszą funkcję przez wszystkie recenzje treningowe
X_train_tokens_imdb = [clean_and_tokenize(doc) for doc in X_train_imdb]
# i przez wszystkie recenzje testowe
X_test_tokens_imdb = [clean_and_tokenize(doc) for doc in X_test_imdb]

# --- PODGLĄD WYNIKU ---
print("\nPrzykład tokenów (pierwsze 30):")
# sprawdzamy, jak wygląda pierwsza recenzja po czyszczeniu i tokenizacji
print(X_train_tokens_imdb[0][:30])

# Zadanie ocena sentymentu opini IMBD

In [ ]:
import os
import re
import numpy as np
import pandas as pd
import gensim
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import time # obliczanie czasu treningu, żeby wiedzieć ile to zajmuje

# importy z TensorFlow/Keras – to tylko do tokenizera i paddingu!
from tf_keras.preprocessing.text import Tokenizer
from tf_keras.preprocessing.sequence import pad_sequences
import kagglehub

# ukrycie komunikatów INFO i WARNING od TensorFlow/oneDNN, żeby nie zaśmiecać konsoli
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# --- KONFIGURACJA STAŁYCH (NASZE HIPERPARAMETRY) ---
MAX_SEQUENCE_LENGTH = 300   # max długość recenzji, do tego będziemy przycinać/wypełniać
EMBEDDING_DIM = 100         # wymiar wektora słowa Word2Vec (każde słowo ma 100 liczb)
W2V_MIN_COUNT = 5           # ignorujemy słowa, które występują rzadziej niż 5 razy (bo są za rzadkie)
LSTM_HIDDEN_DIM = 128       # rozmiar warstwy ukrytej LSTM
NUM_CLASSES = 2             # dwie klasy: pozytywny / negatywny
NUM_LAYERS = 3              # ile warstw LSTM nałożymy na siebie
DROPOUT = 0.3               # regularyzacja, żeby model nie przetrenował się na pamięć
LEARNING_RATE = 0.001       # jak szybko model się uczy
EPOCHS = 5                  # ile razy przelecimy przez cały zbiór danych
BATCH_SIZE = 64             # ile recenzji naraz wrzucamy do modelu
TEST_SIZE = 0.2             # 20% danych zostawiamy na test

# --- DEFINICJE KLAS PYTORCH (NASZ MODEL I SZTUCZKI) ---

class Attention(nn.Module):
    """
    to jest warstwa Attention (uwagi), decyduje, które słowa są najważniejsze w recenzji
    """
    def __init__(self, hidden_dim):
        super(Attention, self).__init__()
        # warstwa liniowa, która ma się nauczyć, ile "wagi" dać każdemu słowu
        self.attention_layer = nn.Linear(hidden_dim, 1, bias=False)

    def forward(self, lstm_outputs):
        # lstm_outputs shape: (batch_size, seq_len, hidden_dim) - stany ukryte z każdego słowa

        # obliczamy wstępną "ważność" każdego słowa
        attention_weights = self.attention_layer(lstm_outputs).squeeze(2)

        # ZASTOSOWANIE SOFTMAX: to klucz, żeby wagi sumowały się do 1, czyli dostajemy % ważności każdego słowa
        soft_attention_weights = torch.softmax(attention_weights, dim=1)

        # obliczamy ważoną sumę stanów ukrytych, to jest nasz "wektor kontekstu"
        # ten wektor reprezentuje całą recenzję, z naciskiem na słowa z wysoką wagą
        context_vector = torch.bmm(soft_attention_weights.unsqueeze(1), lstm_outputs).squeeze(1)

        return context_vector

class EmbeddingClassifier(nn.Module):
    """
    główna architektura: word2vec embeddings -> lstm -> attention -> klasyfikator
    """
    def __init__(self, embedding_dim, lstm_hidden_dim, num_layers, num_classes, dropout=0.3):
        super(EmbeddingClassifier, self).__init__()

        # definicja warstwy LSTM. przyjmuje wektory (100D) i ma 3 warstwy.
        self.lstm = nn.LSTM(input_size=embedding_dim, hidden_size=lstm_hidden_dim, num_layers=num_layers,
                              batch_first=True, dropout=dropout, bidirectional=False) # WAŻNE: używamy tylko zwykłego LSTM

        # dodajemy warstwę uwagi, która "ogarnie" wyjścia z LSTM
        self.attention = Attention(lstm_hidden_dim)

        # warstwa klasyfikująca: przyjmuje wektor kontekstu (output attention) i przewiduje 2 klasy
        self.fc = nn.Linear(lstm_hidden_dim, num_classes)

    def forward(self, x):
        # x to są nasze wektory słów (embeddingi)
        lstm_outputs, (hidden, cell) = self.lstm(x) # przepuszczamy przez LSTM

        # wszystkie stany ukryte lecą do Attention, żeby wygenerować wektor kontekstu
        context_vector = self.attention(lstm_outputs)

        # wektor kontekstu idzie do klasyfikatora
        out = self.fc(context_vector)
        return out

class TextEmbeddingDataset(Dataset):
    """
    customowy dataset pyTorch, który zamienia ID słów na gotowe wektory Word2Vec.
    dzięki niemu model dostaje od razu wektory, a nie ID.
    """
    def __init__(self, sequences_padded, labels, embedding_matrix):
        self.sequences = sequences_padded
        self.labels = torch.LongTensor(labels)
        # macierz embeddingów przerabiamy na tensor PyTorch
        self.embedding_matrix = torch.from_numpy(embedding_matrix)

    def __len__(self):
        return len(self.sequences) # zwraca ile mamy recenzji

    def __getitem__(self, idx):
        indices = self.sequences[idx] # pobieramy ID słów dla recenzji

        # KLUCZOWY KROK: zamieniamy te ID na wektory z naszej macierzy Word2Vec
        embeddings = torch.index_select(self.embedding_matrix, 0, torch.LongTensor(indices))

        # zwracamy wektory (X) i etykietę (Y)
        return embeddings, self.labels[idx]

# --- KROK 1: Pobieranie i wczytywanie danych IMDb z Kaggle ---
print("pobieranie zbioru danych IMDb z Kaggle Hub...")
# używamy kagglehub do pobrania gotowego pliku csv
path = kagglehub.dataset_download("lakshmi25npathi/imdb-dataset-of-50k-movie-reviews")
csv_file_path = os.path.join(path, "IMDB Dataset.csv")
# ładujemy dane do pandas
df = pd.read_csv(csv_file_path)
print("pobieranie i wczytywanie zakończone.")
print(f"wczytano {len(df)} recenzji.")

# upraszczanie etykiet
all_reviews_text = df['review'].tolist()
# zamieniamy 'positive' i 'negative' na 1 i 0
all_labels = df['sentiment'].apply(lambda x: 1 if x == 'positive' else 0).values

# podział na zbiór treningowy i testowy (80/20)
X_train_text, X_test_text, Y_train, Y_test = train_test_split(
    all_reviews_text, all_labels, test_size=TEST_SIZE, random_state=42, stratify=all_labels)

# --- KROK 2: Trenowanie Word2Vec (tworzenie embeddingów) ---
# prosta tokenizacja na potrzeby gensim (usuwamy <br /> i interpunkcję, wszystko małe litery)
tokenized_corpus = [
    re.sub(r'[^a-z\s]', '', re.sub(r'<br />', ' ', review).lower()).split()
    for review in all_reviews_text]

print("rozpoczynanie trenowania modelu Word2Vec...")
start_w2v_time = time.time()
# trenujemy Word2Vec w trybie Skip-gram (sg=1), żeby dostać 100D wektory
w2v_model = gensim.models.Word2Vec(
    sentences=tokenized_corpus,
    vector_size=EMBEDDING_DIM,
    window=5,
    min_count=W2V_MIN_COUNT,
    workers=4,
    sg=1 # skip-gram
)
end_w2v_time = time.time()
print(f"trenowanie Word2Vec zakończone. wymiar embeddingów: {EMBEDDING_DIM}")
print(f"czas trenowania Word2Vec: {(end_w2v_time - start_w2v_time):.2f} sekund.")

# --- KROK 3: Przygotowanie danych do PyTorch (tokenizacja i padding) ---
keras_tokenizer = Tokenizer()
# tworzymy słownik ID dla wszystkich słów
keras_tokenizer.fit_on_texts(all_reviews_text)
word_index = keras_tokenizer.word_index
VOCAB_SIZE = len(word_index) + 1 # +1 na padding (ID 0)

# tworzymy macierz, która będzie przechowywać wszystkie wektory Word2Vec (VOCAB_SIZE wierszy, EMBEDDING_DIM kolumn)
embedding_matrix = np.zeros((VOCAB_SIZE, EMBEDDING_DIM), dtype=np.float32)

# wypełnianie macierzy naszymi wytrenowanymi wektorami
for word, i in word_index.items():
    if word in w2v_model.wv:
        embedding_matrix[i] = w2v_model.wv[word]

# zamieniamy teksty na listy ID słów
X_train_sequences = keras_tokenizer.texts_to_sequences(X_train_text)
X_test_sequences = keras_tokenizer.texts_to_sequences(X_test_text)

# PADDING: wyrównujemy długość wszystkich recenzji do 300
X_train_padded = pad_sequences(X_train_sequences, maxlen=MAX_SEQUENCE_LENGTH, padding='post', truncating='post')
X_test_padded = pad_sequences(X_test_sequences, maxlen=MAX_SEQUENCE_LENGTH, padding='post', truncating='post')

# ustawienia DataLoadera
pin_memory = torch.cuda.is_available() # włączenie optymalizacji dla GPU, jeśli jest

# tworzenie obiektów DataLoader, które będą podawały dane w batchach do modelu
train_dataset = TextEmbeddingDataset(X_train_padded, Y_train, embedding_matrix)
test_dataset = TextEmbeddingDataset(X_test_padded, Y_test, embedding_matrix)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, pin_memory=pin_memory)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, pin_memory=pin_memory)

# --- KROK 4: Funkcje Treningu i Ewaluacji ---

def train_model(model, loader, criterion, optimizer, device):
    """funkcja do trenowania modelu przez jedną epokę"""
    model.train() # włącza tryb treningowy (włącza dropout)
    total_loss = 0
    correct = 0
    total = 0

    if device.type == 'cuda':
        torch.cuda.empty_cache() # czyścimy cache GPU na wszelki wypadek

    for inputs, labels in loader:
        # dane lecą na GPU/CPU
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad() # ZEROWANIE GRADIENTÓW (musowe w PyTorch)
        outputs = model(inputs) # forward pass (przewidywanie)
        loss = criterion(outputs, labels) # obliczenie błędu

        loss.backward() # backward pass (obliczenie gradientów)
        optimizer.step() # aktualizacja wag

        total_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    avg_loss = total_loss / len(loader)
    accuracy = 100 * correct / total
    return avg_loss, accuracy

def evaluate_model(model, loader, criterion, device):
    """funkcja do mierzenia wydajności (bez nauki)"""
    model.eval() # wyłącza tryb treningowy (wyłącza dropout)
    total_loss = 0
    correct = 0
    total = 0

    with torch.no_grad(): # wyłącza liczenie gradientów (oszczędność czasu/pamięci)
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            total_loss += loss.item()

            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_loss = total_loss / len(loader)
    accuracy = 100 * correct / total
    return avg_loss, accuracy

# --- KROK 5: Inicjalizacja i Pętla Treningowa ---
# sprawdzamy czy mamy GPU, inaczej używamy CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"używane urządzenie: {device}")

# inicjalizujemy nasz model i wysyłamy go na wybrane urządzenie
model = EmbeddingClassifier(
    embedding_dim=EMBEDDING_DIM,
    lstm_hidden_dim=LSTM_HIDDEN_DIM,
    num_layers=NUM_LAYERS,
    num_classes=NUM_CLASSES,
    dropout=DROPOUT,
   ).to(device)

# funkcja straty (błędu)
criterion = nn.CrossEntropyLoss()
# optymalizator Adam, który aktualizuje wszystkie wagi modelu z naszym learning_rate
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

print("\nrozpoczynanie treningu PyTorch LSTM...")
start_lstm_time = time.time()

# pętla treningowa, przechodzimy przez 5 epok
for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()

    # trening na danych treningowych
    train_loss, train_acc = train_model(model, train_loader, criterion, optimizer, device)
    # ewaluacja na danych testowych (jak sobie radzi na nowych danych)
    test_loss, test_acc = evaluate_model(model, test_loader, criterion, device)

    epoch_end_time = time.time()
    epoch_duration = epoch_end_time - epoch_start_time

    print(f"epoka {epoch}/{EPOCHS} (czas trwania: {epoch_duration:.2f} s)")
    print(f"  trening: loss={train_loss:.4f}, acc={train_acc:.2f}%")
    print(f"  test:    loss={test_loss:.4f}, acc={test_acc:.2f}%")

end_lstm_time = time.time()
total_lstm_time = end_lstm_time - start_lstm_time
print("trening PyTorch zakończony.")
print(f"całkowity czas treningu LSTM (dla {EPOCHS} epok): {total_lstm_time:.2f} sekund.")

# --- KROK 6: Zapisanie Wytrenowanego Modelu ---
# zapisujemy tylko wagi modelu (state_dict), co jest najlepszą praktyką
SAVE_PATH = 'imdb_sentiment_model_weights.pth'
torch.save(model.state_dict(), SAVE_PATH)
print(f"\nwagi modelu zostały pomyślnie zapisane w pliku: {SAVE_PATH}")